# Data Processing

In [1]:
import pandas as pd
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

from sklearn.model_selection import cross_val_score

### Mammogram Dataset

We begin by importing the mammogram data as a dataframe, replacing '?' with NaN. DataFrames have static methods to remove NaN values

In [2]:
mammogram_df = pd.read_csv('mammographic_masses.data.txt', header=0, names=['BI-RADS', 'Age', 'Shape', 'Margin', 'Density', 'Severity'], na_values=['?'])
mammogram_df.head()

,BI-RADS,Age,Shape,Margin,Density,Severity
0,4.0,43.0,1.0,1.0,NaN,1
1,5.0,58.0,4.0,5.0,3.0,1
2,4.0,28.0,1.0,1.0,3.0,0
3,5.0,74.0,1.0,5.0,NaN,1
4,4.0,65.0,1.0,NaN,3.0,0


Off the bat we can see there are missing data points. Lets see the extent of the damage

In [3]:
mammogram_df.describe()

,BI-RADS,Age,Shape,Margin,Density,Severity
count,958.000000,955.000000,929.000000,912.000000,884.000000,960.000000
mean,4.347599,55.475393,2.721206,2.793860,2.910633,0.462500
std,1.783838,14.482917,1.243428,1.565702,0.380647,0.498852
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


We can see by the count that there are a number of missing data points, especially on the density column.

In [4]:
mammogram_df.dropna(inplace=True)

We want to be careful that we don't introduce bias by simply dropping values! Alternatively, we can interpolate using the mean but this will still add some bias

In [5]:
mammogram_df.describe()

,BI-RADS,Age,Shape,Margin,Density,Severity
count,829.000000,829.000000,829.000000,829.000000,829.000000,829.000000
mean,4.393245,55.768396,2.781665,2.810615,2.915561,0.484922
std,1.889394,14.675456,1.243088,1.566276,0.351136,0.500074
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,46.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


The standard deviation of NaN removed data set is similar to the standard deviation of the entire data set, specifically on the Density column which had the most number of missing data points. Suggests that we won't introduce bias by simply dropping the rows with incomplete data. The count is also identical across all features so the dataset should be nice and clean.

Lets break the data set into Numpy features and labels for later consumption

BI-RADS (col 0) is an assesment of how confident the severity classification is; it is not a "predictive" attribute and so we will discard it.

In [6]:
mammogram_features = mammogram_df.columns[1:-1]
mammogram_x = mammogram_df[mammogram_features].values
mammogram_y = mammogram_df['Severity'].values

Since some models require normalised features, we should go ahead and prepare that now

In [7]:
mammogram_x_scaled = scaler.fit_transform(mammogram_x)